# Kaggle - Retail Data Analytics 

### Context
The Challenge - One challenge of modeling retail data is the need to make decisions based on limited history. Holidays and select major events come once a year, and so does the chance to see how strategic decisions impacted the bottom line. In addition, markdowns are known to affect sales – the challenge is to predict which departments will be affected and to what extent.

### Content
You are provided with historical sales data for 45 stores located in different regions - each store contains a number of departments. The company also runs several promotional markdown events throughout the year. These markdowns precede prominent holidays, the four largest of which are the Super Bowl, Labor Day, Thanksgiving, and Christmas. The weeks including these holidays are weighted five times higher in the evaluation than non-holiday weeks.

### The Task
    - Predict the department-wide sales for each store for the following year;
    - Model the effects of markdowns on holiday weeks;
    - Provide recommended actions based on the insights drawn, with prioritization placed on largest business 
    impact;

#### Imports

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_log_error,
mean_squared_error,
mean_absolute_error,
median_absolute_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor

## Data Gathering

In [2]:
features_data = pd.read_csv('features-data-set.csv')
sales_data = pd.read_csv('sales-data-set.csv')
stores_data = pd.read_csv('stores-data-set.csv')

In [4]:
features_data.sample(5)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
659,4,06/04/2012,67.69,3.864,11341.17,NaN,73.38,5677.89,7821.95,131.037548,4.308,False
5760,32,11/05/2012,55.40,3.870,17977.87,NaN,26.79,4338.30,3488.39,197.548161,8.090,False
4290,24,03/02/2012,38.61,3.796,65021.23,4189.09,254.01,47452.43,7146.25,137.063258,8.659,False
4139,23,07/09/2012,66.74,3.921,13578.43,91.22,85.23,1288.27,4827.11,138.472936,4.156,True
6383,36,07/05/2010,75.56,2.786,NaN,NaN,NaN,NaN,NaN,209.121587,8.464,False


### Sales
Historical sales data, which covers to 2010-02-05 to 2012-11-01. Within this tab you will find the following fields:

    - Store - the store number;
    - Dept - the department number;
    - Date - the week;
    - Weekly_Sales -  sales for the given department in the given 
    store;
    - IsHoliday - whether the week is a special holiday week;

In [5]:
sales_data.sample(5)

,Store,Dept,Date,Weekly_Sales,IsHoliday
98748,11,7,28/01/2011,18185.74,False
472,1,4,03/12/2010,37110.55,False
260804,27,30,28/01/2011,2974.16,False
268435,28,11,30/07/2010,12168.39,False
243119,25,49,10/02/2012,11396.09,True


### Stores
Anonymized information about the 45 stores, indicating the type and size of store.

In [6]:
stores_data.sample(5)

,Store,Type,Size
8,9,B,125833
6,7,B,70713
4,5,B,34875
23,24,A,203819
29,30,C,42988


## Data Wrangling

In [7]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


In [8]:
features_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


In [4]:
features_data['Date'] = pd.to_datetime(features_data['Date'])
sales_data['Date'] = pd.to_datetime(sales_data['Date'])

In [7]:
store_1_features = features_data.query('Store == 1')
store_1_features.sample(5)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
18,1,2010-11-06,80.43,2.668,NaN,NaN,NaN,NaN,NaN,211.456095,7.808,False
73,1,2011-01-07,85.55,3.524,NaN,NaN,NaN,NaN,NaN,215.184137,7.962,False
155,1,2013-01-25,53.37,3.227,965.89,1097.91,0.10,225.36,1831.88,224.235552,6.525,False
127,1,2012-07-13,77.12,3.256,7218.13,45.20,36.24,3909.38,2596.37,221.924158,6.908,False
65,1,2011-06-05,64.61,3.906,NaN,NaN,NaN,NaN,NaN,215.796004,7.682,False


In [8]:
store_1_sales = sales_data.query('Store == 1')

store1 = pd.merge(store_1_features, store_1_sales, on='Date', how='inner')

store1.head()

,Store_x,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_x,Store_y,Dept,Weekly_Sales,IsHoliday_y
0,1,2010-05-02,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,1,1,24924.50,False
1,1,2010-05-02,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,1,2,50605.27,False
2,1,2010-05-02,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,1,3,13740.12,False
3,1,2010-05-02,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,1,4,39954.04,False
4,1,2010-05-02,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,1,5,32229.38,False


In [9]:
store1.drop(['Store_x', 'IsHoliday_x', 'Store_y'], axis=1, inplace=True)

store1['Year_Week'] = store1['Date'].dt.strftime('%Y%U')

store1['Year_Week'] = store1['Date'].dt.strftime('%Y%U')

store1.head()

,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Dept,Weekly_Sales,IsHoliday_y,Year_Week
0,2010-05-02,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,1,24924.50,False,201018
1,2010-05-02,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,2,50605.27,False,201018
2,2010-05-02,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,3,13740.12,False,201018
3,2010-05-02,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,4,39954.04,False,201018
4,2010-05-02,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,5,32229.38,False,201018


## Forecasting 

With only Feature Engineering and, one initial variable.

In [10]:
data = store1.query('Dept == 1').copy()

In [11]:
data.drop(['Temperature',
           'Fuel_Price',
           'MarkDown1',
           'MarkDown2',
           'MarkDown3',
           'MarkDown4',
           'MarkDown5',
           'CPI',
           'Unemployment',
           'IsHoliday_y',
           'Date'], axis=1, inplace=True)

In [12]:
data.head()

,Dept,Weekly_Sales,Year_Week
0,1,24924.50,201018
73,1,46039.49,201048
145,1,41595.55,201007
218,1,19403.54,201008
290,1,21827.90,201018


### Feature Engineering

In [13]:
data2 = data.copy()

data2['Last_Week_Sales'] = data2['Weekly_Sales'].shift()

data2['Last_Week_Diff'] = data2['Weekly_Sales'].diff()

data2 = data2.dropna()

data2.head()

,Dept,Weekly_Sales,Year_Week,Last_Week_Sales,Last_Week_Diff
73,1,46039.49,201048,24924.50,21114.99
145,1,41595.55,201007,46039.49,-4443.94
218,1,19403.54,201008,41595.55,-22192.01
290,1,21827.90,201018,19403.54,2424.36
363,1,21043.39,201048,21827.90,-784.51


In [14]:
data2['Year_Week'] = data2['Year_Week'].astype(int)

In [15]:
data2.drop_duplicates(subset='Year_Week', inplace=True)

### Setting a baseline

#### Mean Squared Log Error - https://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-log-error

In [16]:
week_range = data2['Year_Week'].values

# Train Test need df with at least one value
week_range = np.delete(week_range, week_range.argmin())

In [17]:
week_range.sort()
week_range

array([201005, 201006, 201007, 201008, 201010, 201011, 201012, 201014,
       201015, 201016, 201017, 201018, 201019, 201020, 201021, 201023,
       201024, 201025, 201027, 201028, 201029, 201030, 201032, 201033,
       201034, 201035, 201036, 201037, 201038, 201040, 201041, 201042,
       201043, 201044, 201046, 201047, 201048, 201049, 201050, 201051,
       201052, 201101, 201102, 201103, 201104, 201106, 201107, 201108,
       201110, 201111, 201112, 201113, 201114, 201115, 201116, 201117,
       201119, 201120, 201121, 201123, 201124, 201125, 201126, 201128,
       201129, 201130, 201131, 201132, 201133, 201134, 201136, 201137,
       201138, 201139, 201140, 201141, 201142, 201143, 201144, 201145,
       201146, 201147, 201149, 201150, 201151, 201152, 201201, 201202,
       201203, 201204, 201205, 201207, 201208, 201209, 201210, 201211,
       201212, 201213, 201214, 201215, 201216, 201217, 201219, 201220,
       201221, 201222, 201223, 201224, 201225, 201226, 201228, 201229,
      

In [54]:
mean_error = []

for week in week_range:
    
        train = data2.query('Year_Week < @week')
        test = data2.query('Year_Week == @week')

        p = test['Last_Week_Sales'].values
        
        error = np.sqrt(mean_squared_log_error(test['Weekly_Sales'].values, p))

        print(week, error)

        mean_error.append(error)
    
    

201005 0.7806805450807186
201006 0.0002706354991097726
201007 0.10150399558044221
201008 0.7625101718265199
201010 0.027963298582026397
201011 0.0506453109044287
201012 0.16963247274699178
201014 0.11949194397111285
201015 0.8925309229398302
201016 0.08608896830175183
201017 0.025061200047883148
201018 0.11772772061055647
201019 0.08330011237169188
201020 0.24775698431601967
201021 0.05320823400764141
201023 0.12948714842213427
201024 0.025649775467718072
201025 0.006910036685043863
201027 0.050573240524908414
201028 0.030880484605990688
201029 0.11024433218097762
201030 0.0013901779708298534
201032 0.1194884299243757
201033 0.013027011251802278
201034 0.0034081717414178314
201035 0.2872784358522882
201036 0.07972854511457861
201037 0.061775166442334495
201038 0.06575212495895322
201040 0.11353899840144877
201041 0.1428042788933599
201042 0.05466736657881732
201043 0.41393393500039366
201044 0.05384526527903688
201046 0.0001764514994402333
201047 0.038183010670730155
201048 0.613629850

In [55]:
np.mean(mean_error)

0.17107278434027337

### Fitting the Models

#### Random Forest

In [56]:
mean_error_rf = []

for week in week_range:
        
        train = data2.query('Year_Week < @week')
        test = data2.query('Year_Week == @week')
        
        X_train = train.drop(['Weekly_Sales'], axis=1)
        X_test = test.drop(['Weekly_Sales'], axis=1)
        
        y_train = train['Weekly_Sales'].values
        y_test = test['Weekly_Sales'].values
        
        model = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=0)
        model.fit(X_train, y_train)

        p = model.predict(X_test)
        
        error = np.sqrt(mean_squared_log_error(y_test, p))

        #print(week, error)

        mean_error_rf.append(error)

In [57]:
np.mean(mean_error_rf)

0.10558520424405819

#### LGBM Regressor

In [68]:
mean_error_lgbm = []

for week in week_range:
        
        train = data2.query('Year_Week < @week')
        test = data2.query('Year_Week == @week')
        
        X_train = train.drop(['Weekly_Sales'], axis=1)
        X_test = test.drop(['Weekly_Sales'], axis=1)
        
        y_train = train['Weekly_Sales'].values
        y_test = test['Weekly_Sales'].values
        
        model = LGBMRegressor(n_estimators=100,
                              n_jobs=-1,
                              random_state=0,
                              boosting_type= "gbdt")
        
        model.fit(X_train, y_train)

        p = model.predict(X_test)
        
        error = np.sqrt(mean_squared_log_error(y_test, p))

        #print(week, error)

        mean_error_lgbm.append(error)
        

In [69]:
np.mean(mean_error_lgbm)

0.2086972636168177

### Model Export

In [25]:
sample = data2.copy()

In [27]:
X = sample.drop(['Weekly_Sales'], axis=1)
y = sample['Weekly_Sales']

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42) 

In [29]:
X_test.to_csv("X_test.csv", index = False)

In [29]:
model = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [30]:
import pickle

pickle_out = open('model.pkl', 'wb')
pickle.dump(model, pickle_out)
pickle_out.close()